## 一、从零到一手动搭建RAG系统

In [4]:
!pip install PyPDF2 markdown tqdm tiktoken bs4 IPython

  Using cached markdown-3.10-py3-none-any.whl.metadata (5.1 kB)
  Using cached beautifulsoup4-4.14.3-py3-none-any.whl.metadata (3.8 kB)
Using cached markdown-3.10-py3-none-any.whl (107 kB)
Using cached beautifulsoup4-4.14.3-py3-none-any.whl (107 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [bs4]2/4 [beautifulsoup4]


In [7]:
!pip install html2text

In [8]:
import os
from openai import OpenAI
import matplotlib.pyplot as plt
import numpy as np
from typing import Dict, List, Optional, Tuple, Union

import PyPDF2
import markdown
import html2text
import json
from tqdm import tqdm
import tiktoken
import re
from bs4 import BeautifulSoup
from IPython.display import display, Code, Markdown

In [11]:
from dotenv import load_dotenv
load_dotenv(dotenv_path="/home/crq/llm_scripts/langchain/.env")
api_key = os.getenv("OPENAI_API_KEY")

In [12]:
# 实例化客户端
client = OpenAI(api_key=api_key,
                base_url="https://api.chatanywhere.tech")

##### 1.OpenAI Embedding 模型简介

##### OpenAI Embedding模型获取方法

In [16]:
# 调用 embedding API 获取文本的向量表示
response = client.embeddings.create(
    input="我是秦始皇",  # 输入文本
    model="text-embedding-3-small"  # 选择 Embedding 模型
)

In [17]:
# 打印返回的 embedding 向量
print(response.data[0].embedding)

[0.012142355553805828, -0.006939774379134178, -0.014905663207173347, 0.03189956024289131, -0.02315058372914791, -0.0038479294162243605, -0.037732210010290146, 0.028227150440216064, -0.06084679067134857, 0.008456443436443806, -0.007569844834506512, 0.023060573264956474, 0.010882214643061161, 0.011692305095493793, 0.011611295863986015, 0.003580149495974183, -0.07712060958147049, -0.008586958050727844, 0.04597713053226471, 0.022862551733851433, -0.022988565266132355, 0.0035778991878032684, 0.024284711107611656, -0.0013028953690081835, -0.00136927783023566, 0.0199282243847847, 0.009622073732316494, -0.01337549276649952, 0.027381055057048798, -0.04763331636786461, -0.00656173238530755, -0.033123698085546494, 0.010945221409201622, 0.010909217409789562, -0.013645523227751255, -0.03170153871178627, -0.01361851952970028, -0.03744417801499367, -0.02997334487736225, -0.05397002398967743, -0.0014626632910221815, -0.05519415810704231, 0.027741095051169395, 0.02019825391471386, 0.0026597967371344566

In [18]:
len(response.data[0].embedding)

1536

返回的 Embedding 向量可以直接用于多种应用场景，例如存储在向量数据库中，进行文本相似度搜索等。默认情况下，`text-embedding-3-small` 生成的向量长度为 1536，`text-embedding-3-large` 的向量长度为 3072。

- 余弦相似度介绍与效果简介

- 欧氏距离与余弦相似度计算公式

- 余弦相似度计算函数

In [19]:
def cosine_sim(vector1: List[float], vector2: List[float]) -> float:
    """
    计算两个向量之间的余弦相似度
    """
    dot_product = np.dot(vector1, vector2)
    magnitude = np.linalg.norm(vector1) * np.linalg.norm(vector2)
    if not magnitude:
        return 0
    return dot_product / magnitude

In [20]:
text1 = '我喜欢吃苹果'
text2 = "苹果是我最喜欢吃的水果"
text3 = "我喜欢用苹果手机"

In [21]:
vector1 = client.embeddings.create(
    input=text1,  
    model="text-embedding-3-large"  
).data[0].embedding

vector2 = client.embeddings.create(
    input=text2,  
    model="text-embedding-3-large"  
).data[0].embedding

vector3 = client.embeddings.create(
    input=text3,  
    model="text-embedding-3-large"  
).data[0].embedding

In [22]:
cosine_sim(vector1, vector2)

np.float64(0.7661800739633343)

In [23]:
cosine_sim(vector1, vector3)

np.float64(0.7224817543294454)

In [24]:
cosine_sim(vector2, vector3)

np.float64(0.6315437708887437)

&emsp;&emsp;为了实现 RAG 模型的功能，我们首先需要一个向量化（Embedding）模块。向量化是 RAG 的基础，它的作用是将文档片段转化为向量表示，便于后续的检索操作。在这个过程中，我们将实现一个向量化类，用来将文本片段映射成向量。

&emsp;&emsp;为了便于扩展和未来可能使用不同的模型，我们首先编写一个 **Embedding 基类**。该基类定义了获取文本向量表示的方法，同时包含一个计算两个向量之间**余弦相似度**的功能。这样，如果我们未来使用不同的向量化模型，只需继承该基类并重写向量获取的逻辑，而不需要重复编写相似度计算部分。

In [25]:
class BaseEmbeddings:
    """
    向量化的基类，用于将文本转换为向量表示。不同的子类可以实现不同的向量获取方法。
    """
    def __init__(self, path: str, is_api: bool) -> None:
        """
        初始化基类。
        
        参数：
        path (str) - 如果是本地模型，path 表示模型路径；如果是API模式，path可以为空
        is_api (bool) - 表示是否使用API调用，如果为True表示通过API获取Embedding
        """
        self.path = path
        self.is_api = is_api
    
    def get_embedding(self, text: str, model: str) -> List[float]:
        """
        抽象方法，用于获取文本的向量表示，具体实现需要在子类中定义。
        
        参数：
        text (str) - 需要转换为向量的文本
        model (str) - 所使用的模型名称
        
        返回：
        list[float] - 文本的向量表示
        """
        raise NotImplementedError
    
    @classmethod
    def cosine_similarity(cls, vector1: List[float], vector2: List[float]) -> float:
        """
        计算两个向量之间的余弦相似度，用于衡量它们的相似程度。
        
        参数：
        vector1 (list[float]) - 第一个向量
        vector2 (list[float]) - 第二个向量
        
        返回：
        float - 余弦相似度值，范围从 -1 到 1，越接近 1 表示向量越相似
        """
        dot_product = np.dot(vector1, vector2)  # 向量点积
        magnitude = np.linalg.norm(vector1) * np.linalg.norm(vector2)  # 向量的模
        if not magnitude:
            return 0
        return dot_product / magnitude  # 计算余弦相似度


我们在这个基类基础上，可以通过继承它来实现具体的模型。例如，我们可以使用 OpenAI 的 API 来生成文本的向量表示，只需重写 `get_embedding` 方法即可。

In [72]:
class OpenAIEmbedding(BaseEmbeddings):
    """
    使用 OpenAI 的 Embedding API 来获取文本向量的类，继承自 BaseEmbeddings。
    """
    def __init__(self, path: str = '', is_api: bool = True) -> None:
        """
        初始化类，设置 OpenAI API 客户端，如果使用的是 API 调用。
        
        参数：
        path (str) - 本地模型的路径，使用API时可以为空
        is_api (bool) - 是否通过 API 获取 Embedding，默认为 True
        """
        super().__init__(path, is_api)
        if self.is_api:
            # 初始化 OpenAI API 客户端
            from openai import OpenAI
            self.client = OpenAI()
            self.client.api_key = os.getenv("OPENAI_API_KEY")  # 从环境变量中获取 API 密钥
            self.client.base_url = os.getenv("OPENAI_BASE_URL")  # 从环境变量中获取 API 基础URL
    
    def get_embedding(self, text: str, model: str = "text-embedding-3-large") -> List[float]:
        """
        使用 OpenAI 的 Embedding API 获取文本的向量表示。
        
        参数：
        text (str) - 需要转化为向量的文本
        model (str) - 使用的 Embedding 模型名称，默认为 'text-embedding-3-large'
        
        返回：
        list[float] - 文本的向量表示
        """
        print("self.client.api_key:", self.client.api_key)
        print("self.client.base_url:", self.client.base_url)
        if self.is_api:
            # 去掉文本中的换行符，保证输入格式规范
            text = text.replace("\n", " ")
            # 调用 OpenAI API 获取文本的向量表示
            return self.client.embeddings.create(input=[text], model=model).data[0].embedding
        else:
            raise NotImplementedError  # 如果不是 API 模式，这里未实现本地模型的处理


🚗使用千问的embedding模型

In [87]:
from langchain_community.embeddings import DashScopeEmbeddings
from dotenv import load_dotenv

load_dotenv(dotenv_path="/home/crq/llm_scripts/langchain/.env")

embeddings = DashScopeEmbeddings(
    model="text-embedding-v4", dashscope_api_key=os.getenv("DASHSCOPE_API_KEY")
)

text = "This is a test document."

query_result = embeddings.embed_query(text)
print(query_result)

doc_results = embeddings.embed_documents(["foo"])
print(doc_results)

[-0.038538604974746704, 0.03440577909350395, 0.026333853602409363, -0.046597618609666824, -0.05496659129858017, -0.01645381562411785, -0.012773016467690468, 0.044376224279403687, -0.06524699926376343, 0.12873753905296326, 0.004181257914751768, 0.0032949603628367186, -0.011681691743433475, 0.012630950659513474, -0.006935399491339922, -0.012340361252427101, 0.0030915478710085154, -0.0640588104724884, 0.004010132979601622, 0.005624518264085054, -0.04623599350452423, 0.02319548837840557, -0.0238024964928627, 0.009350519627332687, -0.01554975938051939, 0.009763802401721478, -0.0293947272002697, -0.08591112494468689, 0.019953802227973938, -0.008388346061110497, -0.0488448403775692, 0.023789580911397934, 0.007342224474996328, 0.00339020905084908, -0.04357548803091049, 0.038254473358392715, 0.016208428889513016, 0.03921018913388252, -0.016867097467184067, 0.0042619770392775536, -0.036653004586696625, 0.005660034716129303, -0.06571193784475327, 0.025662269443273544, -0.012146634981036186, 0.006

In [88]:
class QwenEmbedding(BaseEmbeddings):
    """
    使用 Qwen 的 Embedding API 来获取文本向量的类，继承自 BaseEmbeddings。
    """
    def __init__(self, path: str = '', is_api: bool = True) -> None:
        """
        初始化类，设置 OpenAI API 客户端，如果使用的是 API 调用。
        
        参数：
        path (str) - 本地模型的路径，使用API时可以为空
        is_api (bool) - 是否通过 API 获取 Embedding，默认为 True
        """
        super().__init__(path, is_api)
        if self.is_api:
            # 初始化 OpenAI API 客户端
            from openai import OpenAI
            self.client = OpenAI()
            self.client.api_key = os.getenv("DASHSCOPE_API_KEY")  # 从环境变量中获取 API 密钥
            self.client.base_url = os.getenv("DASHSCOPE_BASE_URL")  # 从环境变量中获取 API 基础URL
    
    def get_embedding(self, text: str, model: str = "text-embedding-v4") -> List[float]:
        """
        使用 OpenAI 的 Embedding API 获取文本的向量表示。
        
        参数：
        text (str) - 需要转化为向量的文本
        model (str) - 使用的 Embedding 模型名称，默认为 'text-embedding-3-large'
        
        返回：
        list[float] - 文本的向量表示
        """
        print("self.client.api_key:", self.client.api_key)
        print("self.client.base_url:", self.client.base_url)
        if self.is_api:
            # 去掉文本中的换行符，保证输入格式规范
            text = text.replace("\n", " ")
            # 调用 OpenAI API 获取文本的向量表示
            return self.client.embeddings.create(input=[text], model=model).data[0].embedding
        else:
            raise NotImplementedError  # 如果不是 API 模式，这里未实现本地模型的处理


这样设计的结构让我们可以轻松替换或扩展向量化模型，而不需要改变整体框架。

- 使用示例

In [73]:
# 初始化 Embedding 模型
embedding_model = OpenAIEmbedding()

In [90]:
# 初始化 Embedding 模型
qwen_embedding_model = QwenEmbedding()

In [29]:
# 输入需要获取向量表示的文本
text = "这是一个示例文本，用于演示 OpenAI Embedding 的使用。"

# 获取文本的向量表示
embedding_vector = embedding_model.get_embedding(text, model="text-embedding-3-large")

print("文本的向量表示为：", embedding_vector)

文本的向量表示为： [0.02648121863603592, 0.013051084242761135, -0.007090849336236715, -0.010436945594847202, 0.01947532780468464, 0.013410528190433979, -0.0019606035202741623, 0.03531700372695923, 0.001083233393728733, -0.026468146592378616, -0.020873891189694405, 0.03463732823729515, -0.004346004221588373, -0.04122495651245117, 0.008103827945888042, 0.0026239410508424044, 0.03489874303340912, 0.02705632895231247, -0.017959129065275192, -0.004254509694874287, 0.010855208151042461, 0.016273008659482002, -0.020638620480895042, -0.00878350343555212, 0.04080669581890106, -0.021292153745889664, 0.014011779800057411, 0.04156479611992836, -0.007299980614334345, 0.02395857498049736, -0.0015374398790299892, -0.03299042209982872, -0.0015750181628391147, -0.03923821076750755, 0.005084498319774866, 0.014639172703027725, 0.044832468032836914, 0.06169365718960762, -0.012502115219831467, 0.003669596277177334, 0.040362291038036346, -0.01856037974357605, 0.0015660320641472936, -0.026742631569504738, -0.01648214

In [93]:
# 输入需要获取向量表示的文本
text = "这是一个示例文本，用于演示 OpenAI Embedding 的使用。"

# 获取文本的向量表示
embedding_vector = qwen_embedding_model.get_embedding(text, model="text-embedding-v4")

print("文本的向量表示为：", embedding_vector)

self.client.api_key: sk-c4d40f85a9e24d76a5d0a44255150594
self.client.base_url: https://dashscope.aliyuncs.com/compatible-mode/v1/
文本的向量表示为： [-0.06656274199485779, 0.019207844510674477, 0.018718861043453217, -0.010467281565070152, 0.0015166098019108176, -0.06589039415121078, 0.025396527722477913, 0.04183856397867203, -0.026557860895991325, 0.12420155107975006, -0.005890711210668087, -0.025472931563854218, 0.01387487817555666, -0.028605476021766663, -0.040982846170663834, -0.021438825875520706, 0.020766476169228554, -0.06194796785712242, 0.006070259027183056, 0.008862807415425777, -0.001338971545919776, -0.024479685351252556, 0.003035129513591528, -0.023822616785764694, -0.021056808531284332, 0.009710886515676975, 0.03051556460559368, -0.011292439885437489, 0.02055254578590393, -0.0163809135556221, -0.029002774506807327, 0.02854435332119465, 0.07823719829320908, 0.0112313162535429, 0.01830628141760826, 0.06809081137180328, -0.013783194124698639, 0.03786558285355568, -0.01259893923997879,

In [30]:
vector1 = embedding_model.get_embedding(text1)
vector2 = embedding_model.get_embedding(text2)

In [31]:
similarity = BaseEmbeddings.cosine_similarity(vector1, vector2)

In [32]:
print(f"两段文本的余弦相似度为: {similarity}")

两段文本的余弦相似度为: 0.7661800739633343


### 3. 文档加载与切分模块创建

&emsp;&emsp;在实现了向量化之后，我们接下来需要编写一个**文档加载与切分模块**，用于处理不同格式的文档并将其切分为小片段。为什么要进行切分呢？这是为了确保每个文档片段都尽量保持简短且信息集中，以便于后续的向量化和检索。

#### 3.1 文档格式处理函数

&emsp;&emsp;我们的目标是支持多种格式的文档，例如 PDF、Markdown、TXT 等。每种文件格式都有不同的读取方式，下面我们展示一个支持多种格式的简单实现：

In [33]:
def read_file_content(cls, file_path: str):
    # 根据文件扩展名选择读取方法
    if file_path.endswith('.pdf'):
        return cls.read_pdf(file_path)
    elif file_path.endswith('.md'):
        return cls.read_markdown(file_path)
    elif file_path.endswith('.txt'):
        return cls.read_text(file_path)
    else:
        raise ValueError("Unsupported file type")

#### 3.2 文档切分函数

&emsp;&emsp;这里我们考虑将文档按**Token**长度进行切分，设置一个最大的 Token 长度，然后按这个长度进行切分。在这个过程中，我们也会确保每个片段之间有一定的重叠，避免重要信息被切掉。

<center><center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20231218182814731.png" alt="image-20231218182814731" style="zoom:33%;" /></center> 

In [34]:
def get_chunk(cls, text: str, max_token_len: int = 600, cover_content: int = 150):
    chunk_text = []
    curr_len = 0
    curr_chunk = ''
    lines = text.split('\n')  # 以换行符为单位切分文本

    for line in lines:
        line = line.replace(' ', '')
        line_len = len(enc.encode(line))  # 计算当前行的 Token 长度
        if line_len > max_token_len:
            print('warning line_len = ', line_len)
        if curr_len + line_len <= max_token_len:
            curr_chunk += line + '\n'
            curr_len += line_len + 1
        else:
            chunk_text.append(curr_chunk)
            curr_chunk = curr_chunk[-cover_content:] + line
            curr_len = line_len + cover_content

    if curr_chunk:
        chunk_text.append(curr_chunk)

    return chunk_text

完整类编写如下：

In [35]:
enc = tiktoken.get_encoding("cl100k_base")

In [36]:
len(enc.encode("你好，好久不见！"))

9

In [37]:
class ReadFiles:
    """
    读取文件的类，用于从指定路径读取支持的文件类型（如 .txt、.md、.pdf）并进行内容分割。
    """

    def __init__(self, path: str) -> None:
        """
        初始化函数，设定要读取的文件路径，并获取该路径下所有符合要求的文件。
        :param path: 文件夹路径
        """
        self._path = path
        self.file_list = self.get_files()  # 获取文件列表

    def get_files(self):
        """
        遍历指定文件夹，获取支持的文件类型列表（txt, md, pdf）。
        :return: 文件路径列表
        """
        file_list = []
        for filepath, dirnames, filenames in os.walk(self._path):
            # os.walk 函数将递归遍历指定文件夹
            for filename in filenames:
                # 根据文件后缀筛选支持的文件类型
                if filename.endswith(".md"):
                    file_list.append(os.path.join(filepath, filename))
                elif filename.endswith(".txt"):
                    file_list.append(os.path.join(filepath, filename))
                elif filename.endswith(".pdf"):
                    file_list.append(os.path.join(filepath, filename))
        return file_list

    def get_content(self, max_token_len: int = 600, cover_content: int = 150):
        """
        读取文件内容并进行分割，将长文本切分为多个块。
        :param max_token_len: 每个文档片段的最大 Token 长度
        :param cover_content: 在每个片段之间重叠的 Token 长度
        :return: 切分后的文档片段列表
        """
        docs = []
        for file in self.file_list:
            content = self.read_file_content(file)  # 读取文件内容
            # 分割文档为多个小块
            chunk_content = self.get_chunk(content, max_token_len=max_token_len, cover_content=cover_content)
            docs.extend(chunk_content)
        return docs

    @classmethod
    def get_chunk(cls, text: str, max_token_len: int = 600, cover_content: int = 150):
        """
        将文档内容按最大 Token 长度进行切分。
        :param text: 文档内容
        :param max_token_len: 每个片段的最大 Token 长度
        :param cover_content: 重叠的内容长度
        :return: 切分后的文档片段列表
        """
        chunk_text = []
        curr_len = 0
        curr_chunk = ''
        token_len = max_token_len - cover_content
        lines = text.splitlines()  # 以换行符分割文本为行

        for line in lines:
            line = line.replace(' ', '')  # 去除空格
            line_len = len(enc.encode(line))  # 计算当前行的 Token 长度
            if line_len > max_token_len:
                # 如果单行长度超过限制，将其分割为多个片段
                num_chunks = (line_len + token_len - 1) // token_len
                for i in range(num_chunks):
                    start = i * token_len
                    end = start + token_len
                    # 防止跨单词分割
                    while not line[start:end].rstrip().isspace():
                        start += 1
                        end += 1
                        if start >= line_len:
                            break
                    curr_chunk = curr_chunk[-cover_content:] + line[start:end]
                    chunk_text.append(curr_chunk)
                start = (num_chunks - 1) * token_len
                curr_chunk = curr_chunk[-cover_content:] + line[start:end]
                chunk_text.append(curr_chunk)
            elif curr_len + line_len <= token_len:
                # 当前片段长度未超过限制时，继续累加
                curr_chunk += line + '\n'
                curr_len += line_len + 1
            else:
                chunk_text.append(curr_chunk)  # 保存当前片段
                curr_chunk = curr_chunk[-cover_content:] + line
                curr_len = line_len + cover_content

        if curr_chunk:
            chunk_text.append(curr_chunk)

        return chunk_text

    @classmethod
    def read_file_content(cls, file_path: str):
        """
        读取文件内容，根据文件类型选择不同的读取方式。
        :param file_path: 文件路径
        :return: 文件内容
        """
        if file_path.endswith('.pdf'):
            return cls.read_pdf(file_path)
        elif file_path.endswith('.md'):
            return cls.read_markdown(file_path)
        elif file_path.endswith('.txt'):
            return cls.read_text(file_path)
        else:
            raise ValueError("Unsupported file type")

    @classmethod
    def read_pdf(cls, file_path: str):
        """
        读取 PDF 文件内容。
        :param file_path: PDF 文件路径
        :return: PDF 文件中的文本内容
        """
        with open(file_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            text = ""
            for page_num in range(len(reader.pages)):
                text += reader.pages[page_num].extract_text()
            return text

    @classmethod
    def read_markdown(cls, file_path: str):
        """
        读取 Markdown 文件内容，并将其转换为纯文本。
        :param file_path: Markdown 文件路径
        :return: 纯文本内容
        """
        with open(file_path, 'r', encoding='utf-8') as file:
            md_text = file.read()
            html_text = markdown.markdown(md_text)
            # 使用 BeautifulSoup 从 HTML 中提取纯文本
            soup = BeautifulSoup(html_text, 'html.parser')
            plain_text = soup.get_text()
            # 使用正则表达式移除网址链接
            text = re.sub(r'http\S+', '', plain_text) 
            return text

    @classmethod
    def read_text(cls, file_path: str):
        """
        读取普通文本文件内容。
        :param file_path: 文本文件路径
        :return: 文件内容
        """
        with open(file_path, 'r', encoding='utf-8') as file:
            return file.read()

In [38]:
class Documents:
    """
    文档类，用于读取已分好类的 JSON 格式文档。
    """
    def __init__(self, path: str = '') -> None:
        self.path = path

    def get_content(self):
        """
        读取 JSON 格式的文档内容。
        :return: JSON 文档的内容
        """
        with open(self.path, mode='r', encoding='utf-8') as f:
            content = json.load(f)
        return content

In [42]:
# 初始化 ReadFiles 类，指定文件目录路径
file_reader = ReadFiles(path="/home/crq/llm_scripts/rag/data")

# 获取目录下所有支持的文件类型
file_list = file_reader.get_files()
print("支持的文件列表：", file_list)

支持的文件列表： ['/home/crq/llm_scripts/rag/data/刑法.pdf']


In [43]:
# 将文件内容读取并分块
document_chunks = file_reader.get_content(max_token_len=600, cover_content=150)
print("分块后的文档内容：", document_chunks)

分块后的文档内容： ['－1－中华人民共和国刑法\n（1979年7月1日第五届全国人民代表大会第二次会\n议通过1997年3月14日第八届全国人民代表大会\n第五次会议修订根据1998年12月29日第九届全\n国人民代表大会常务委员会第六次会议通过的《全国人\n民代表大会常务委员会关于惩治骗购外汇、逃汇和非法\n买卖外汇犯罪的决定》、1999年12月25日第九届全\n国人民代表大会常务委员会第十三次会议通过的《中华\n人民共和国刑法修正案》、2001年8月31日第九届\n全国人民代表大会常务委员会第二十三次会议通过的\n《中华人民共和国刑法修正案（二）》、2001年12\n月29日第九届全国人民代表大会常务委员会第二十五\n次会议通过的《中华人民共和国刑法修正案（三）》、\n2002年12月28日第九届全国人民代表大会常务委员\n会第三十一次会议通过的《中华人民共和国刑法修正案\n（四）》、2005年2月28日第十届全国人民代表大\n会常务委员会第十四次会议通过的《中华人民共和国刑\n', '日第九届全国人民代表大会常务委员会第二十五\n次会议通过的《中华人民共和国刑法修正案（三）》、\n2002年12月28日第九届全国人民代表大会常务委员\n会第三十一次会议通过的《中华人民共和国刑法修正案\n（四）》、2005年2月28日第十届全国人民代表大\n会常务委员会第十四次会议通过的《中华人民共和国刑\n法修正案（五）》、2006年6月29日第十届全国人民代表大会常务委员会第二十二次会议通过的《中华人－2－民共和国刑法修正案（六）》、2009年2月28日第\n十一届全国人民代表大会常务委员会第七次会议通过的\n《中华人民共和国刑法修正案（七）》、2009年8月\n27日第十一届全国人民代表大会常务委员会第十次会\n议通过的《全国人民代表大会常务委员会关于修改部分\n法律的决定》、2011年2月25日第十一届全国人民\n代表大会常务委员会第十九次会议通过的《中华人民共\n和国刑法修正案（八）》、2015年8月29日第十二\n届全国人民代表大会常务委员会第十六次会议通过的\n', '7日第十一届全国人民代表大会常务委员会第十次会\n议通过的《全国人民代表大会常务委员会关于修改部分\n法律的决定》、2011年2月25日第十一届全国人民\n代表大会常务委员会第十九次会议通过的《中

In [44]:
document_chunks[0]

'－1－中华人民共和国刑法\n（1979年7月1日第五届全国人民代表大会第二次会\n议通过1997年3月14日第八届全国人民代表大会\n第五次会议修订根据1998年12月29日第九届全\n国人民代表大会常务委员会第六次会议通过的《全国人\n民代表大会常务委员会关于惩治骗购外汇、逃汇和非法\n买卖外汇犯罪的决定》、1999年12月25日第九届全\n国人民代表大会常务委员会第十三次会议通过的《中华\n人民共和国刑法修正案》、2001年8月31日第九届\n全国人民代表大会常务委员会第二十三次会议通过的\n《中华人民共和国刑法修正案（二）》、2001年12\n月29日第九届全国人民代表大会常务委员会第二十五\n次会议通过的《中华人民共和国刑法修正案（三）》、\n2002年12月28日第九届全国人民代表大会常务委员\n会第三十一次会议通过的《中华人民共和国刑法修正案\n（四）》、2005年2月28日第十届全国人民代表大\n会常务委员会第十四次会议通过的《中华人民共和国刑\n'

In [45]:
document_chunks[1]

'日第九届全国人民代表大会常务委员会第二十五\n次会议通过的《中华人民共和国刑法修正案（三）》、\n2002年12月28日第九届全国人民代表大会常务委员\n会第三十一次会议通过的《中华人民共和国刑法修正案\n（四）》、2005年2月28日第十届全国人民代表大\n会常务委员会第十四次会议通过的《中华人民共和国刑\n法修正案（五）》、2006年6月29日第十届全国人民代表大会常务委员会第二十二次会议通过的《中华人－2－民共和国刑法修正案（六）》、2009年2月28日第\n十一届全国人民代表大会常务委员会第七次会议通过的\n《中华人民共和国刑法修正案（七）》、2009年8月\n27日第十一届全国人民代表大会常务委员会第十次会\n议通过的《全国人民代表大会常务委员会关于修改部分\n法律的决定》、2011年2月25日第十一届全国人民\n代表大会常务委员会第十九次会议通过的《中华人民共\n和国刑法修正案（八）》、2015年8月29日第十二\n届全国人民代表大会常务委员会第十六次会议通过的\n'

### 4. 词向量数据库与向量检索模块

&emsp;&emsp;接下来我们将继续构建**向量数据库**以及**检索模块**，这是 RAG 模型中的核心功能之一。向量数据库用于存储文档片段及其对应的向量表示，而检索模块则根据用户提出的问题（Query）在数据库中检索相关的文档片段。通过这些功能，我们创建的简易 RAG 能够根据输入的查询快速找到最相关的文档片段。

&emsp;&emsp;为了构建这个向量数据库，我们需要以下几个关键功能：

1. **持久化存储（persist）：** 将数据库存储到本地，便于下次加载使用。
2. **加载数据库（load_vector）：** 从本地文件加载已经存储的向量和文档。
3. **获取向量表示（get_vector）：** 将文档转化为向量表示并存储。
4. **检索（query）：** 根据用户的 Query，检索数据库中的相关文档片段。

我们将基于这些功能来实现一个简单的 `VectorStore` 类。

&emsp;&emsp;首先，我们创建一个基础的 `VectorStore` 类，提供上述功能的框架。通过这个类，我们能够将文档片段转化为向量存储，加载本地数据库，进行检索。

In [46]:
class VectorStore:
    def __init__(self, document: List[str] = None) -> None:
        """
        初始化向量存储类，存储文档和对应的向量表示。
        :param document: 文档列表，默认为空。
        """
        if document is None:
            document = []
        self.document = document  # 存储文档内容
        self.vectors = []  # 存储文档的向量表示
    
    def get_vector(self, EmbeddingModel: BaseEmbeddings) -> List[List[float]]:
        """
        使用传入的 Embedding 模型将文档向量化。
        :param EmbeddingModel: 传入的用于生成向量的模型（需继承 BaseEmbeddings 类）。
        :return: 返回文档对应的向量列表。
        """
        # 遍历所有文档，获取每个文档的向量表示
        self.vectors = [EmbeddingModel.get_embedding(doc) for doc in self.document]
        return self.vectors
    
    def persist(self, path: str = 'storage'):
        """
        将文档和对应的向量表示持久化到本地目录中，以便后续加载使用。
        :param path: 存储路径，默认为 'storage'。
        """
        if not os.path.exists(path):
            os.makedirs(path)  # 如果路径不存在，创建路径
        # 保存向量为 numpy 文件
        np.save(os.path.join(path, 'vectors.npy'), self.vectors)
        # 将文档内容存储到文本文件中
        with open(os.path.join(path, 'documents.txt'), 'w') as f:
            for doc in self.document:
                f.write(f"{doc}\n")
    
    def load_vector(self, path: str = 'storage'):
        """
        从本地加载之前保存的文档和向量数据。
        :param path: 存储路径，默认为 'storage'。
        """
        # 加载保存的向量数据
        self.vectors = np.load(os.path.join(path, 'vectors.npy')).tolist()
        # 加载文档内容
        with open(os.path.join(path, 'documents.txt'), 'r') as f:
            self.document = [line.strip() for line in f.readlines()]

    def get_similarity(self, vector1: List[float], vector2: List[float]) -> float:
        """
        计算两个向量的余弦相似度。
        :param vector1: 第一个向量。
        :param vector2: 第二个向量。
        :return: 返回两个向量的余弦相似度，范围从 -1 到 1。
        """
        dot_product = np.dot(vector1, vector2)
        magnitude = np.linalg.norm(vector1) * np.linalg.norm(vector2)
        if not magnitude:
            return 0
        return dot_product / magnitude

    def query(self, query: str, EmbeddingModel: BaseEmbeddings, k: int = 1) -> List[str]:
        """
        根据用户的查询文本，检索最相关的文档片段。
        :param query: 用户的查询文本。
        :param EmbeddingModel: 用于将查询向量化的嵌入模型。
        :param k: 返回最相似的文档数量，默认为 1。
        :return: 返回最相似的文档列表。
        """
        # 将查询文本向量化
        query_vector = EmbeddingModel.get_embedding(query)
        # 计算查询向量与每个文档向量的相似度
        similarities = [self.get_similarity(query_vector, vector) for vector in self.vectors]
        # 获取相似度最高的 k 个文档索引
        top_k_indices = np.argsort(similarities)[-k:][::-1]
        # 返回对应的文档内容
        return [self.document[idx] for idx in top_k_indices]

上述代码解释如下：
- **get_vector 方法：** 这个方法使用传入的 `EmbeddingModel` 对所有文档进行向量化，并将这些向量存储在 `self.vectors` 中。
- **persist 方法：** 该方法将文档片段及其向量表示保存到本地文件系统，便于持久化存储。
- **load_vector 方法：** 从本地文件系统加载已保存的文档片段和向量，供后续检索使用。
- **get_similarity 方法：** 计算两个向量之间的余弦相似度，用于比较查询和文档向量的相似度。
- **query 方法：** 接收用户输入的查询，通过向量化后在数据库中检索最相关的文档片段，并返回最匹配的文档。

假设我们已经有一组文档片段存储在 `documents` 中，并且使用 OpenAI 的 Embedding API 进行向量化处理，以下是一个简化的运行示例：

In [47]:
# 初始化文档列表
documents = [
    "机器学习是人工智能的一个分支。",
    "深度学习是一种特殊的机器学习方法。",
    "监督学习是一种训练模型的方式。",
    "强化学习通过奖励和惩罚进行学习。",
    "无监督学习不依赖标签数据。",
]

In [49]:
# 创建向量数据库
vector_store = VectorStore(document=documents)

# 使用 OpenAI Embedding 模型对文档进行向量化
embedding_model = OpenAIEmbedding()

# 获取文档向量并存储
vector_store.get_vector(embedding_model)

# 持久化存储到本地
vector_store.persist('storage')

In [51]:
# 模拟用户查询
query = "什么是深度学习？"
result = vector_store.query(query, embedding_model)

print("检索结果：", result)

检索结果： ['深度学习是一种特殊的机器学习方法。']


在上面这段代码中，npy 文件是 NumPy 库用于存储数组数据的文件格式。npy 文件能够高效地保存和加载 NumPy 数组，并保留数组的形状、数据类型等信息。它是一种二进制文件格式，用于序列化 NumPy 数组，使得存储和读取过程更加快速和便捷。

In [52]:
loaded_array = np.load('./storage/vectors.npy')

print(loaded_array)

[[ 0.0150427   0.02493685 -0.00436787 ... -0.01719165 -0.01386749
   0.01126523]
 [-0.01286558  0.02198915 -0.00355636 ... -0.01165039 -0.00909946
  -0.0171959 ]
 [-0.0227828   0.0174784  -0.0156523  ... -0.00209008  0.00612738
   0.01555292]
 [-0.0254771   0.01967829 -0.00521657 ... -0.02507718 -0.00610462
  -0.00051681]
 [ 0.02986923  0.00549699 -0.00834242 ... -0.00765595 -0.01394945
   0.01053807]]


### 5. 大模型问答模块编写

&emsp;&emsp;现在我们已经构建了向量数据库和检索模块，接下来我们将实现大模型模块，用于根据检索到的相关文档片段生成对用户问题的回答。为了简化和便于扩展，我们会先实现一个基类 `BaseModel`，然后再以 **GPT4o** 模型为例，展示如何使用大语言模型来完成这个任务。

#### 5.1 大模型模块的基类

&emsp;&emsp;这里我们先编写一个基类 `BaseModel`，它包含两个主要方法：
- `chat`：负责处理用户的输入并生成回答。
- `load_model`：如果是使用本地模型，这个方法负责加载模型。如果使用 API 模型（如 OpenAI），可以不用实现这个方法。

In [53]:
class BaseModel:
    """
    基础模型类，作为所有模型的基类。
    包含一些通用的接口，如加载模型、生成回答等。
    """
    def __init__(self, path: str = '') -> None:
        self.path = path  # 用于存储模型文件的路径，默认为空。

    def chat(self, prompt: str, history: List[dict], content: str) -> str:
        """
        使用模型生成回答的抽象方法。
        :param prompt: 用户的提问内容
        :param history: 之前的对话历史（字典列表）
        :param content: 提供的上下文内容
        :return: 模型生成的答案
        """
        pass  # 具体的实现由子类提供

    def load_model(self):
        """
        加载模型的方法，通常用于本地模型。
        """
        pass  # 如果是 API 模型，可能不需要实现

#### 5.2 借助GPT4o/Qwen3-max模型进行对话

In [62]:
class GPT4oChat(BaseModel):
    """
    基于 GPT-4o 模型的对话类，继承自 BaseModel。
    主要用于通过 OpenAI API 来生成对话回答。
    """
    def __init__(self, api_key: str, base_url: str = "https://api.chatanywhere.tech") -> None:
        """
        初始化 GPT-4o 模型。
        :param api_key: OpenAI API 的密钥
        :param base_url: 用于访问 OpenAI API 的基础 URL，默认为代理 URL
        """
        super().__init__()
        self.client = OpenAI(api_key=api_key, base_url=base_url)  # 初始化 OpenAI 客户端

    def chat(self, prompt: str, history: List = [], content: str = '') -> str:
        """
        使用 GPT-4o 生成回答。
        :param prompt: 用户的提问
        :param history: 之前的对话历史（可选）
        :param content: 可参考的上下文信息（可选）
        :return: 生成的回答
        """
        # 构建包含问题和上下文的完整提示
        full_prompt = PROMPT_TEMPLATE['GPT4o_PROMPT_TEMPLATE'].format(question=prompt, context=content)

        # 调用 GPT-4o 模型进行推理
        response = self.client.chat.completions.create(
            model="gpt-4o-mini",  # 使用 GPT-4o 小型模型
            messages=[
                {"role": "user", "content": full_prompt}
            ]
        )

        # 返回模型生成的第一个回答
        return response.choices[0].message.content

In [94]:
class Qwen3max(BaseModel):
    """
    基于 GPT-4o 模型的对话类，继承自 BaseModel。
    主要用于通过 OpenAI API 来生成对话回答。
    """
    def __init__(self, api_key: str, base_url: str = "https://dashscope.aliyuncs.com/compatible-mode/v1") -> None:
        """
        初始化 Qwen3max 模型。
        :param api_key: DASHSCOPE_API_KEY 密钥
        :param base_url: 用于访问 DASHSCOPE API 的基础 URL，默认为代理 URL
        """
        super().__init__()
        self.client = OpenAI(api_key=api_key, base_url=base_url)  # 初始化 OpenAI 客户端

    def chat(self, prompt: str, history: List = [], content: str = '') -> str:
        """
        使用 GPT-4o 生成回答。
        :param prompt: 用户的提问
        :param history: 之前的对话历史（可选）
        :param content: 可参考的上下文信息（可选）
        :return: 生成的回答
        """
        # 构建包含问题和上下文的完整提示
        full_prompt = PROMPT_TEMPLATE['GPT4o_PROMPT_TEMPLATE'].format(question=prompt, context=content)

        # 调用 qwen3-max 模型进行推理
        response = self.client.chat.completions.create(
            model="qwen3-max",  # 使用 GPT-4o 小型模型
            messages=[
                {"role": "user", "content": full_prompt}
            ]
        )

        # 返回模型生成的第一个回答
        return response.choices[0].message.content

#### 5.3 提示模板

&emsp;&emsp;为了方便维护和复用提示语，可以使用一个字典来保存不同模型的提示模板。在这里我们为 `GPT4o` 定义了一个模板：

In [63]:
PROMPT_TEMPLATE = dict(
    GPT4o_PROMPT_TEMPLATE="""
    下面有一个或许与这个问题相关的参考段落，若你觉得参考段落能和问题相关，则先总结参考段落的内容。
    若你觉得参考段落和问题无关，则使用你自己的原始知识来回答用户的问题，并且总是使用中文来进行回答。
    问题: {question}
    可参考的上下文：
    ···
    {context}
    ···
    有用的回答:"""
)

### 6. RAG Demo完整流程演示

&emsp;&emsp;接下来，我们展示一个完整的 RAG Demo，结合我们前面实现的向量检索和大模型模块，展示如何在实际应用中使用 RAG 模型来回答问题。

In [75]:
# 加载并切分文档
docs = ReadFiles('/home/crq/llm_scripts/rag/data').get_content(max_token_len=600, cover_content=150)
vector = VectorStore(docs)

In [76]:
# 使用 OpenAI Embedding 模型进行向量化
embedding = OpenAIEmbedding()
vector.get_vector(EmbeddingModel=embedding)

self.client.api_key: sk-0n4Wygwpp5zBCbn67fTylHeKOwtgSRsL13OZoc4ONCBAbTPP
self.client.base_url: https://api.chatanywhere.tech
self.client.api_key: sk-0n4Wygwpp5zBCbn67fTylHeKOwtgSRsL13OZoc4ONCBAbTPP
self.client.base_url: https://api.chatanywhere.tech
self.client.api_key: sk-0n4Wygwpp5zBCbn67fTylHeKOwtgSRsL13OZoc4ONCBAbTPP
self.client.base_url: https://api.chatanywhere.tech
self.client.api_key: sk-0n4Wygwpp5zBCbn67fTylHeKOwtgSRsL13OZoc4ONCBAbTPP
self.client.base_url: https://api.chatanywhere.tech
self.client.api_key: sk-0n4Wygwpp5zBCbn67fTylHeKOwtgSRsL13OZoc4ONCBAbTPP
self.client.base_url: https://api.chatanywhere.tech
self.client.api_key: sk-0n4Wygwpp5zBCbn67fTylHeKOwtgSRsL13OZoc4ONCBAbTPP
self.client.base_url: https://api.chatanywhere.tech
self.client.api_key: sk-0n4Wygwpp5zBCbn67fTylHeKOwtgSRsL13OZoc4ONCBAbTPP
self.client.base_url: https://api.chatanywhere.tech
self.client.api_key: sk-0n4Wygwpp5zBCbn67fTylHeKOwtgSRsL13OZoc4ONCBAbTPP
self.client.base_url: https://api.chatanywhere.tech


RateLimitError: Error code: 429 - {'error': {'message': 'text-embedding-3-large模型免费API限制每日200次请求，请00:00后再试，如有更多需求，请访问 https://api.chatanywhere.tech/#/shop 购买付费API。The free account is limited to 200 requests per day. Please try again after 00:00 the next day. If you have additional requirements, please visit https://api.chatanywhere.tech/#/shop to purchase a premium key.(当前请求使用的ApiKey: sk-0n4****bTPP)【如果您遇到问题，欢迎加入QQ群咨询：1003520025】', 'type': 'chatanywhere_error', 'param': None, 'code': '429 TOO_MANY_REQUESTS'}}

In [95]:
# 使用 OpenAI Embedding 模型进行向量化
qwen_embedding = QwenEmbedding()
vector.get_vector(EmbeddingModel=qwen_embedding)

self.client.api_key: sk-c4d40f85a9e24d76a5d0a44255150594
self.client.base_url: https://dashscope.aliyuncs.com/compatible-mode/v1/
self.client.api_key: sk-c4d40f85a9e24d76a5d0a44255150594
self.client.base_url: https://dashscope.aliyuncs.com/compatible-mode/v1/
self.client.api_key: sk-c4d40f85a9e24d76a5d0a44255150594
self.client.base_url: https://dashscope.aliyuncs.com/compatible-mode/v1/
self.client.api_key: sk-c4d40f85a9e24d76a5d0a44255150594
self.client.base_url: https://dashscope.aliyuncs.com/compatible-mode/v1/
self.client.api_key: sk-c4d40f85a9e24d76a5d0a44255150594
self.client.base_url: https://dashscope.aliyuncs.com/compatible-mode/v1/
self.client.api_key: sk-c4d40f85a9e24d76a5d0a44255150594
self.client.base_url: https://dashscope.aliyuncs.com/compatible-mode/v1/
self.client.api_key: sk-c4d40f85a9e24d76a5d0a44255150594
self.client.base_url: https://dashscope.aliyuncs.com/compatible-mode/v1/
self.client.api_key: sk-c4d40f85a9e24d76a5d0a44255150594
self.client.base_url: https://das

[[0.0013921504141762853,
  0.005324953235685825,
  0.005914653185755014,
  -0.0148166473954916,
  0.00010637547529768199,
  0.07231766730546951,
  0.04254312813282013,
  0.06683734804391861,
  -0.009745935909450054,
  0.016779959201812744,
  -0.014675402082502842,
  -0.029266057536005974,
  -0.04093293100595474,
  0.00995074212551117,
  0.037091054022312164,
  -0.048221196979284286,
  -0.007839121855795383,
  -0.06463391333818436,
  -0.011334947310388088,
  -0.05130034685134888,
  -1.337969570158748e-05,
  0.031356487423181534,
  0.03889899700880051,
  0.04943590611219406,
  0.01406804658472538,
  0.04454881325364113,
  0.07214817404747009,
  -0.0457635261118412,
  -0.021102068945765495,
  0.013206449337303638,
  -0.051724083721637726,
  -0.10067975521087646,
  0.024294216185808182,
  0.014576530084013939,
  0.0178251750767231,
  0.028814071789383888,
  -0.04847543686628342,
  0.03813626989722252,
  0.011172514408826828,
  0.012118859216570854,
  0.008171048946678638,
  -0.025932664051

In [96]:
# 将向量和文档保存到本地
vector.persist(path='storage')

In [101]:
# 用户提出问题
question = '什么是故意犯罪'

In [106]:
# 在数据库中检索最相关的文档片段
content = vector.query(question, EmbeddingModel=qwen_embedding, k=2)

self.client.api_key: sk-c4d40f85a9e24d76a5d0a44255150594
self.client.base_url: https://dashscope.aliyuncs.com/compatible-mode/v1/


In [108]:
content[0]

'种结果发生，因而构成犯罪的，是故意犯罪。\n故意犯罪，应当负刑事责任。\n第十五条应当预见自己的行为可能发生危害社会的结果，\n因为疏忽大意而没有预见，或者已经预见而轻信能够避免，以致\n发生这种结果的，是过失犯罪。\n过失犯罪，法律有规定的才负刑事责任。\n第十六条行为在客观上虽然造成了损害结果，但是不是出\n于故意或者过失，而是由于不能抗拒或者不能预见的原因所引起的，不是犯罪。\n第十七条已满十六周岁的人犯罪，应当负刑事责任。\n已满十四周岁不满十六周岁的人，犯故意杀人、故意伤害致\n人重伤或者死亡、强奸、抢劫、贩卖毒品、放火、爆炸、投放危\n险物质罪的，应当负刑事责任。\n已满十二周岁不满十四周岁的人，犯故意杀人、故意伤害罪，\n致人死亡或者以特别残忍手段致人重伤造成严重残疾，情节恶劣，\n经最高人民检察院核准追诉的，应当负刑事责任。\n'

In [107]:
content[1]

'、制造条件的，是犯罪预\n备。\n对于预备犯，可以比照既遂犯从轻、减轻处罚或者免除处罚。\n第二十三条已经着手实行犯罪，由于犯罪分子意志以外的\n原因而未得逞的，是犯罪未遂。\n对于未遂犯，可以比照既遂犯从轻或者减轻处罚。\n第二十四条在犯罪过程中，自动放弃犯罪或者自动有效地\n防止犯罪结果发生的，是犯罪中止。\n对于中止犯，没有造成损害的，应当免除处罚；造成损害的，应当减轻处罚。\n第三节共同犯罪－11－第二十五条共同犯罪是指二人以上共同故意犯罪。\n二人以上共同过失犯罪，不以共同犯罪论处；应当负刑事责\n任的，按照他们所犯的罪分别处罚。\n第二十六条组织、领导犯罪集团进行犯罪活动的或者在共\n同犯罪中起主要作用的，是主犯。\n三人以上为共同实施犯罪而组成的较为固定的犯罪组织，是\n犯罪集团。\n对组织、领导犯罪集团的首要分子，按照集团所犯的全部罪\n行处罚。\n'

In [105]:
# 使用 GPT4oChat 模型生成答案
chat = GPT4oChat(api_key = os.getenv("OPENAI_API_KEY"))  # 传入 OpenAI API 密钥
print(chat.chat(question, [], content))

参考段落主要讲述了如何在JupyterLab中选择特定的Python虚拟环境作为Kernel。用户需要打开一个新的notebook，然后在右上角的Kernel选项中选择相应的虚拟环境（如“Python(myenv)”）。这样，notebook就会在所选的虚拟环境中运行。此外，段落还提到完成工作后应该如何退出虚拟环境。

针对问题“如何选择Jupyter的Kernel”，可以总结为以下步骤：

1. 打开Jupyter Notebook或JupyterLab。
2. 创建一个新的notebook。
3. 在右上角找到Kernel选项。
4. 从下拉菜单中选择想要使用的Kernel，例如选择对应的虚拟环境。
5. 如果需要，完成工作后记得退出虚拟环境。

遵循这些步骤，你就可以成功选择所需的Kernel来运行你的notebook了。


In [109]:
# 使用 qwen3-max 模型生成答案
qwen_chat = Qwen3max(api_key = os.getenv("DASHSCOPE_API_KEY"))  # 传入 API 密钥
print(qwen_chat.chat(question, [], content))

参考段落与问题“什么是故意犯罪”密切相关，以下是对参考段落相关内容的总结：

根据提供的法律条文（主要来自《中华人民共和国刑法》第十四条），**故意犯罪**是指行为人**明知自己的行为会发生危害社会的结果，并且希望或者放任这种结果发生**，因而构成犯罪的情形。对于故意犯罪，行为人**应当负刑事责任**。

此外，段落还通过对比说明了：
- **过失犯罪**（第十五条）：指应当预见而未预见，或已预见但轻信能够避免，导致危害结果发生；只有法律有规定时才负刑事责任。
- **非出于故意或过失的行为**（第十六条）：如因不可抗力或不能预见的原因造成损害，不构成犯罪。
- 在共同犯罪中（第二十五条），明确指出“共同犯罪是指二人以上共同故意犯罪”，进一步强调“故意”是成立共同犯罪的前提。

综上，**故意犯罪的核心在于行为人主观上对危害结果具有“明知”并“希望或放任”其发生的心态**，这是其区别于过失犯罪和意外事件的关键。


- 完整函数封装

In [106]:
def run_mini_rag(question: str, knowledge_base_path: str, k: int = 1) -> str:
    """
    运行一个简化版的RAG项目。
    
    :param question: 用户提出的问题
    :param knowledge_base_path: 知识库的路径，包含文档的文件夹路径
    :param api_key: OpenAI API密钥，用于调用GPT-4o模型
    :param k: 返回与问题最相关的k个文档片段，默认为1
    :return: 返回GPT-4o模型生成的回答
    """
    api_key = os.getenv("OPENAI_API_KEY")
    # 1. 加载并切分文档
    docs = ReadFiles(knowledge_base_path).get_content(max_token_len=600, cover_content=150)
    vector = VectorStore(docs)

    # 2. 使用 OpenAI Embedding 模型进行向量化
    embedding = OpenAIEmbedding()
    vector.get_vector(EmbeddingModel=embedding)

    # 3. 将向量和文档保存到本地（可选）
    vector.persist(path='storage')

    # 4. 在数据库中检索最相关的文档片段
    content = vector.query(question, EmbeddingModel=embedding, k=k)[0]

    # 5. 使用 GPT-4o 生成答案
    chat = GPT4oChat(api_key=api_key)
    answer = chat.chat(question, [], content)
    
    return answer

In [110]:
def run_mini_rag(question: str, knowledge_base_path: str, k: int = 1) -> str:
    """
    运行一个简化版的RAG项目。
    
    :param question: 用户提出的问题
    :param knowledge_base_path: 知识库的路径，包含文档的文件夹路径
    :param api_key: OpenAI API密钥，用于调用GPT-4o模型
    :param k: 返回与问题最相关的k个文档片段，默认为1
    :return: 返回GPT-4o模型生成的回答
    """
    api_key = os.getenv("DASHSCOPE_API_KEY")
    # 1. 加载并切分文档
    docs = ReadFiles(knowledge_base_path).get_content(max_token_len=600, cover_content=150)
    vector = VectorStore(docs)

    # 2. 使用 OpenAI Embedding 模型进行向量化
    embedding = QwenEmbedding()
    vector.get_vector(EmbeddingModel=embedding)

    # 3. 将向量和文档保存到本地（可选）
    vector.persist(path='storage')

    # 4. 在数据库中检索最相关的文档片段
    content = vector.query(question, EmbeddingModel=embedding, k=k)[0]

    # 5. 使用 GPT-4o 生成答案
    chat = Qwen3max(api_key=api_key)
    answer = chat.chat(question, [], content)
    
    return answer

In [111]:
question = '刑法的任务是什么'

In [112]:
knowledge_base_path = '/home/crq/llm_scripts/rag/data'

In [113]:
answer = run_mini_rag(question, knowledge_base_path)

self.client.api_key: sk-c4d40f85a9e24d76a5d0a44255150594
self.client.base_url: https://dashscope.aliyuncs.com/compatible-mode/v1/
self.client.api_key: sk-c4d40f85a9e24d76a5d0a44255150594
self.client.base_url: https://dashscope.aliyuncs.com/compatible-mode/v1/
self.client.api_key: sk-c4d40f85a9e24d76a5d0a44255150594
self.client.base_url: https://dashscope.aliyuncs.com/compatible-mode/v1/
self.client.api_key: sk-c4d40f85a9e24d76a5d0a44255150594
self.client.base_url: https://dashscope.aliyuncs.com/compatible-mode/v1/
self.client.api_key: sk-c4d40f85a9e24d76a5d0a44255150594
self.client.base_url: https://dashscope.aliyuncs.com/compatible-mode/v1/
self.client.api_key: sk-c4d40f85a9e24d76a5d0a44255150594
self.client.base_url: https://dashscope.aliyuncs.com/compatible-mode/v1/
self.client.api_key: sk-c4d40f85a9e24d76a5d0a44255150594
self.client.base_url: https://dashscope.aliyuncs.com/compatible-mode/v1/
self.client.api_key: sk-c4d40f85a9e24d76a5d0a44255150594
self.client.base_url: https://das

In [114]:
display(Markdown(answer))

参考段落与问题相关，其内容直接阐述了刑法的任务。

总结参考段落内容：  
该段落引自《中华人民共和国刑法》第二条（虽未明确标出条文号，但内容对应），明确指出刑法的任务是：同一切犯罪行为作斗争，以保卫国家安全、人民民主专政的政权和社会主义制度；保护国有财产、集体财产以及公民私人财产；保护公民的人身权利、民主权利和其他合法权益；维护社会秩序和经济秩序；保障社会主义建设事业的顺利进行。

## 二、❌基于LangChain的RAG系统Demo示例--老版本langchain，不予实现

&emsp;&emsp;理解了在`LangChain`中构建`RAG`的基本原理后，我们就可以开始动手实践了。接下来的案例中，我们通过 `Streamlit` 前端界面，结合 `LangChain` 框架 与 `DashScope` 向量嵌入服务，实现了一个轻量化的 `RAG（Retrieval-Augmented Generation）` 智能问答系统，支持上传多个 `PDF` 文档，系统将自动完成文本提取、分块、向量化，并构建基于 `FAISS` 的检索数据库。用户随后可以在页面中输入任意问题，系统会调用大语言模型（如 `DeepSeek-Chat`）对 `PDF` 内容进行语义理解和回答生成。

&emsp;&emsp;其完整代码如下所示：

In [115]:
! pip install streamlit PyPDF2 dashscope faiss-cpu

  Using cached blinker-1.9.0-py3-none-any.whl.metadata (1.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 3.0 MB/s  0:00:03m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 795.4/795.4 kB 3.1 MB/s  0:00:00 eta 0:00:01
Using cached blinker-1.9.0-py3-none-any.whl (8.5 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 3.5 MB/s  0:00:01m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 2.9 MB/s  0:00:08m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11/11 [streamlit]11 [streamlit]


```python
    import streamlit as st
    from PyPDF2 import PdfReader
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    from langchain_core.prompts import ChatPromptTemplate
    from langchain_community.vectorstores import FAISS
    from langchain.tools.retriever import create_retriever_tool
    from langchain.agents import AgentExecutor, create_tool_calling_agent
    from langchain_community.embeddings import DashScopeEmbeddings
    from langchain.chat_models import init_chat_model
    import os
    from dotenv import load_dotenv 
    load_dotenv(override=True)


    DeepSeek_API_KEY = os.getenv("DEEPSEEK_API_KEY")
    dashscope_api_key = os.getenv("dashscope_api_key")

    os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"


    embeddings = DashScopeEmbeddings(
        model="text-embedding-v1", dashscope_api_key=dashscope_api_key
    )

    def pdf_read(pdf_doc):
        text = ""
        for pdf in pdf_doc:
            pdf_reader = PdfReader(pdf)
            for page in pdf_reader.pages:
                text += page.extract_text()
        return text


    def get_chunks(text):
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
        chunks = text_splitter.split_text(text)
        return chunks

    def vector_store(text_chunks):
        vector_store = FAISS.from_texts(text_chunks, embedding=embeddings)
        vector_store.save_local("faiss_db")

    def get_conversational_chain(tools, ques):
        llm = init_chat_model("deepseek-chat", model_provider="deepseek")
        prompt = ChatPromptTemplate.from_messages([
            (
                "system",
                """你是AI助手，请根据提供的上下文回答问题，确保提供所有细节，如果答案不在上下文中，请说"答案不在上下文中"，不要提供错误的答案""",
            ),
            ("placeholder", "{chat_history}"),
            ("human", "{input}"),
            ("placeholder", "{agent_scratchpad}"),
        ])
        
        tool = [tools]
        agent = create_tool_calling_agent(llm, tool, prompt)
        agent_executor = AgentExecutor(agent=agent, tools=tool, verbose=True)
        
        response = agent_executor.invoke({"input": ques})
        print(response)
        st.write("🤖 回答: ", response['output'])

    def check_database_exists():
        """检查FAISS数据库是否存在"""
        return os.path.exists("faiss_db") and os.path.exists("faiss_db/index.faiss")

    def user_input(user_question):
        # 检查数据库是否存在
        if not check_database_exists():
            st.error("❌ 请先上传PDF文件并点击'Submit & Process'按钮来处理文档！")
            st.info("💡 步骤：1️⃣ 上传PDF → 2️⃣ 点击处理 → 3️⃣ 开始提问")
            return
        
        try:
            # 加载FAISS数据库
            new_db = FAISS.load_local("faiss_db", embeddings, allow_dangerous_deserialization=True)
            
            retriever = new_db.as_retriever()
            retrieval_chain = create_retriever_tool(retriever, "pdf_extractor", "This tool is to give answer to queries from the pdf")
            get_conversational_chain(retrieval_chain, user_question)
            
        except Exception as e:
            st.error(f"❌ 加载数据库时出错: {str(e)}")
            st.info("请重新处理PDF文件")

    def main():
        st.set_page_config("🤖 LangChain B站公开课 By九天Hector")
        st.header("🤖 LangChain B站公开课 By九天Hector")
        
        # 显示数据库状态
        col1, col2 = st.columns([3, 1])
        
        with col1:
            if check_database_exists():
            pass
            else:
                st.warning("⚠️ 请先上传并处理PDF文件")
        
        with col2:
            if st.button("🗑️ 清除数据库"):
                try:
                    import shutil
                    if os.path.exists("faiss_db"):
                        shutil.rmtree("faiss_db")
                    st.success("数据库已清除")
                    st.rerun()
                except Exception as e:
                    st.error(f"清除失败: {e}")

        # 用户问题输入
        user_question = st.text_input("💬 请输入问题", 
                                    placeholder="例如：这个文档的主要内容是什么？",
                                    disabled=not check_database_exists())

        if user_question:
            if check_database_exists():
                with st.spinner("🤔 AI正在分析文档..."):
                    user_input(user_question)
            else:
                st.error("❌ 请先上传并处理PDF文件！")

        # 侧边栏
        with st.sidebar:
            st.title("📁 文档管理")
            
            # 显示当前状态
            if check_database_exists():
                st.success("✅ 数据库状态：已就绪")
            else:
                st.info("📝 状态：等待上传PDF")
            
            st.markdown("---")
            
            # 文件上传
            pdf_doc = st.file_uploader(
                "📎 上传PDF文件", 
                accept_multiple_files=True,
                type=['pdf'],
                help="支持上传多个PDF文件"
            )
            
            if pdf_doc:
                st.info(f"📄 已选择 {len(pdf_doc)} 个文件")
                for i, pdf in enumerate(pdf_doc, 1):
                    st.write(f"{i}. {pdf.name}")
            
            # 处理按钮
            process_button = st.button(
                "🚀 提交并处理", 
                disabled=not pdf_doc,
                use_container_width=True
            )
            
            if process_button:
                if pdf_doc:
                    with st.spinner("📊 正在处理PDF文件..."):
                        try:
                            # 读取PDF内容
                            raw_text = pdf_read(pdf_doc)
                            
                            if not raw_text.strip():
                                st.error("❌ 无法从PDF中提取文本，请检查文件是否有效")
                                return
                            
                            # 分割文本
                            text_chunks = get_chunks(raw_text)
                            st.info(f"📝 文本已分割为 {len(text_chunks)} 个片段")
                            
                            # 创建向量数据库
                            vector_store(text_chunks)
                            
                            st.success("✅ PDF处理完成！现在可以开始提问了")
                            st.balloons()
                            st.rerun()
                            
                        except Exception as e:
                            st.error(f"❌ 处理PDF时出错: {str(e)}")
                else:
                    st.warning("⚠️ 请先选择PDF文件")
            
            # 使用说明
            with st.expander("💡 使用说明"):
                st.markdown("""
                **步骤：**
                1. 📎 上传一个或多个PDF文件
                2. 🚀 点击"Submit & Process"处理文档
                3. 💬 在主页面输入您的问题
                4. 🤖 AI将基于PDF内容回答问题
                
                **提示：**
                - 支持多个PDF文件同时上传
                - 处理大文件可能需要一些时间
                - 可以随时清除数据库重新开始
                """)

    if __name__ == "__main__":
        main()
```

基于此，我们能够实现：

* **LangChain 的多模块能力**（向量搜索 + Agent工具）
* **Streamlit 前端交互**
* **FAISS 向量数据库**
* **DashScope Embedding + DeepSeek 模型接入**
* 并完成了完整的 RAG（检索增强生成）流程

以下是各部分功能实现代码讲解：

#### 🔧 1. 导入库 & 环境初始化

```python
import streamlit as st
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
...
load_dotenv(override=True)
```

* `Streamlit` 用于构建网页界面。
* `PyPDF2` 用来读取 PDF 文本。
* `load_dotenv()` 加载 `.env` 中的 API Key，例如：

  ```dotenv
  DEEPSEEK_API_KEY=sk-xxx
  DASHSCOPE_API_KEY=xxx
  ```

---

#### 🔐 2. 加载 API 密钥与设置环境变量

```python
DeepSeek_API_KEY = os.getenv("DEEPSEEK_API_KEY")
dashscope_api_key = os.getenv("dashscope_api_key")
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
```

* 从环境变量中读取 DashScope 和 DeepSeek API。
* 设置 `KMP_DUPLICATE_LIB_OK` 避免某些 MKL 多线程报错。

---

#### 🧠 3. 初始化向量 Embedding 模型

```python
embeddings = DashScopeEmbeddings(
    model="text-embedding-v1", dashscope_api_key=dashscope_api_key
)
```

* 用阿里云 DashScope 提供的 `text-embedding-v1` 将文本转为向量表示，用于相似度搜索。

---

#### 📄 4. 处理 PDF 文本与向量化逻辑

```python
def pdf_read(pdf_doc):
    ...
def get_chunks(text):
    ...
def vector_store(text_chunks):
    ...
```

* `pdf_read`：逐页读取 PDF 内容并拼接。
* `get_chunks`：将长文本切片为多个段落（chunk），每段 1000 字，重叠 200 字。
* `vector_store`：用 FAISS 建立向量索引，并保存到本地 `faiss_db/`。

---

#### 🔁 5. Agent对话链 + 工具调用（核心 RAG）

```python
def get_conversational_chain(tools, ques):
    llm = init_chat_model("deepseek-chat", model_provider="deepseek")
    ...
    agent_executor = AgentExecutor(...)
    response = agent_executor.invoke({"input": ques})
    ...
```

* 初始化 DeepSeek 模型为 Agent。
* 使用 LangChain 的 `create_tool_calling_agent` 构造 Agent，输入：

  * prompt（你设定的系统角色）
  * 工具（retriever 工具）
* `AgentExecutor.invoke`：LangChain 自动判断是否调用工具，完成“读取上下文 → 查询 → 回答”流程。

---

#### 🔍 6. 用户提问逻辑（调用 FAISS）

```python
def user_input(user_question):
    ...
    new_db = FAISS.load_local("faiss_db", embeddings, ...)
    retriever = new_db.as_retriever()
    retrieval_chain = create_retriever_tool(retriever, "pdf_extractor", ...)
    get_conversational_chain(retrieval_chain, user_question)
```

* 加载本地 FAISS 向量库；
* 将其转为 LangChain 的检索工具；
* 交由 Agent 调用完成回答。

---

#### 🧠 7. 检查数据库是否存在

```python
def check_database_exists():
    return os.path.exists("faiss_db") and os.path.exists("faiss_db/index.faiss")
```

简单检查本地是否已有向量化数据。

---

#### 🌐 8. 主界面逻辑（Streamlit）

```python
def main():
    st.set_page_config(...)
    ...
```

* 页面标题与界面配置。
* `st.columns` 分栏：左边显示提示，右边放置“清空数据库”按钮。
* 主输入框：`st.text_input("请输入问题")`

  * 只有当数据库存在时才能提问。
* 侧边栏：

  * PDF 上传器；
  * 提交按钮（处理上传的 PDF → 分片 → 向量化 → 存储）。

---

#### 🎯 9. 提交 PDF 后执行的逻辑

```python
if process_button:
    raw_text = pdf_read(pdf_doc)
    ...
    text_chunks = get_chunks(raw_text)
    vector_store(text_chunks)
```

* 当点击“提交并处理”后：

  1. 读取上传的 PDF；
  2. 切片文本；
  3. 向量化入库；
  4. 弹出气球提示，并 `st.rerun()` 刷新页面状态。

---

#### 📎 项目结构总结

| 模块                | 说明                             |
| ----------------- | ------------------------------ |
| 🧾 PDF解析          | 读取用户上传的 PDF                    |
| ✂️ 文本切片           | 按段落分割内容                        |
| 📊 向量化            | DashScope Embedding + FAISS 建库 |
| 🔁 查询接口           | 用户输入 → 召回相关 chunk              |
| 🤖 DeepSeek Agent | 调用检索工具并给出回答                    |
| 💻 UI层            | Streamlit 实现全部交互               |

&emsp;&emsp;其中LangChain RAG核心功能相关代码如下：

- Step 1：PDF 文件上传与文本提取

&emsp;&emsp;使用 `st.file_uploader()` 组件支持多文件上传，并通过 `PyPDF2.PdfReader` 对每页内容进行提取，组合为整体文本。

```python
    def pdf_read(pdf_doc):
        text = ""
        for pdf in pdf_doc:
            pdf_reader = PdfReader(pdf)
            for page in pdf_reader.pages:
                text += page.extract_text()
        return text
```

- Step 2：文本分块与向量数据库构建

&emsp;&emsp;使用 `RecursiveCharacterTextSplitter` 将长文档切割为固定长度（1000字）+ 重叠（200字）的小块，将文本块通过 `DashScopeEmbeddings` 嵌入为向量，使用 `FAISS` 本地存储向量数据库。


```python
    chunks = text_splitter.split_text(text)
    vector_store = FAISS.from_texts(chunks, embedding=embeddings)
    vector_store.save_local("faiss_db")
```


- Step 3：用户提问与语义检索

&emsp;&emsp;通过 `Streamlit` 获取用户输入问题，如果向量数据库存在，则加载 `FAISS` 检索器，使用 `create_retriever_tool()` 构建 `LangChain` 工具，交由 `AgentExecutor` 执行，自动调用检索器并生成答案。

```python
    retrieval_chain = create_retriever_tool(retriever, ...)
    agent = create_tool_calling_agent(llm, [retrieval_chain], prompt)
    response = agent_executor.invoke({"input": ques})
```

&emsp;&emsp;项目运行效果如下所示：

In [2]:
from IPython.display import Video

In [3]:
Video("https://ml2022.oss-cn-hangzhou.aliyuncs.com/LangChain%20RAG.mp4", width=800, height=400)